# Load Data

First, some technicalities 

In [ ]:
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [ ]:
# Enable pandas conversion for R data frames
pandas2ri.activate()

#### Source my R functions that handle Seurat objects

In [17]:
try:
    # Source the R script
    robjects.r.source("../scripts/data_loader.R")
    print("R script sourced successfully!")
except Exception as e:
    print("Error sourcing R script:", e)

R script sourced successfully!


In [35]:
load_seurat = robjects.globalenv["load_seurat"]
extract_raw_counts = robjects.globalenv["extract_raw_counts"]
load_cell_types = robjects.globalenv["load_cell_types"]

#### Load Seurat file (not public data) with R function

In [ ]:
%%capture

seurat_obj = load_seurat("../data/raw/SeuratProject.h5Seurat")

Load raw RNA-seq counts table (and transform to pandas)

In [ ]:
raw_counts_r = extract_raw_counts(seurat_obj)
# Convert the R data frame to a pandas DataFrame
raw_counts = pandas2ri.rpy2py_dataframe(raw_counts_r)
# print(raw_counts.head())

Load cell type annotations for all cells in raw counts

In [ ]:
cell_types_r = load_cell_types(seurat_obj)
cell_types = pandas2ri.rpy2py_dataframe(cell_types_r)
# print(cell_types.head())

### Save transformed data as CSV for later use 

In [43]:
raw_counts.to_csv("../data/raw_counts.csv")
cell_types.to_csv("../data/cell_types_annotations.csv")